In [1]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)

In [2]:
tickers_map = {
    # Energy
    'CL': 'CL=F',   # WTI Crude Oil
    'NG': 'NG=F',   # Natural Gas
    'RB': 'RB=F',   # Gasoline
    'HO': 'HO=F',   # Heating Oil
    
    # Metals
    'GC': 'GC=F',   # Gold
    'SI': 'SI=F',   # Silver
    'HG': 'HG=F',   # Copper
    'PL': 'PL=F',   # Platinum
    
    # Agriculture
    'ZC': 'ZC=F',   # Corn
    'ZW': 'ZW=F',   # Wheat
    'ZS': 'ZS=F',   # Soybeans
    'KC': 'KC=F',   # Coffee
    'SB': 'SB=F',   # Sugar
    'CT': 'CT=F',   # Cotton
    
    # Livestock
    'LE': 'LE=F',   # Live Cattle
    'HE': 'HE=F',   # Lean Hogs
}

In [3]:
from turtle import window_height
from quanta.clients.yfinance import YahooFinanceClient
from datetime import datetime, timedelta
import polars as pl

initial = 1000
window_days = 30*12  # 12 months
from_date = datetime.now() - timedelta(days=window_days)
to_date = datetime.now() - timedelta(days=1)

timeframe = "1h"

yh = YahooFinanceClient()
df_cl = yh.get_price(
    "CL=F",
    from_date=from_date.strftime("%Y-%m-%d"),
    to_date=to_date.strftime("%Y-%m-%d"),
    interval=timeframe, 
    postclean=True
)
df_rb = yh.get_price(
    "NG=F",
    from_date=from_date.strftime("%Y-%m-%d"),
    to_date=to_date.strftime("%Y-%m-%d"),
    interval=timeframe,
    postclean=True
)

# add symbol column
df_cl = df_cl.with_columns(pl.lit("CL=F").alias("symbol"))
df_rb = df_rb.with_columns(pl.lit("RB=F").alias("symbol"))

# append df_cl and df_rb
df = df_cl.vstack(df_rb)

In [4]:
def create_trades_with_sizing(df_with_signals, initial_capital=1.0, position_size=0.52):
    """
    Version avec gestion de capital constante
    """
    trades_list = []
    position = 0
    entry_idx = None
    trade_num = 0
    capital = initial_capital
    
    df = df_with_signals.with_row_count('idx')
    
    for row in df.iter_rows(named=True):
        current_signal = row['signal']
        
        # Changement de position
        if current_signal != position:
            # Ferme position existante
            if position != 0 and entry_idx is not None:
                exit_price = row['close']
                entry_row = df.filter(pl.col('idx') == entry_idx).to_dicts()[0]
                entry_price = entry_row['close']
                
                # Calcul PnL
                price_change = (exit_price - entry_price) / entry_price
                pnl = price_change * position * position_size
                capital += pnl
                
                # Trade de sortie
                trades_list.append({
                    'timestamp': row['datetime'],
                    'position_number': trade_num,
                    'action': 'SELL' if position == 1 else 'BUY',
                    'price': exit_price,
                    'quantity_usd': position_size,
                    'position_size': position_size,
                    'pnl': pnl,
                    'cumulative_capital': capital
                })
                trade_num += 1
            
            # Ouvre nouvelle position si signal non-zero
            if current_signal != 0:
                trades_list.append({
                    'timestamp': row['datetime'],
                    'position_number': trade_num,
                    'action': 'BUY' if current_signal == 1 else 'SELL',
                    'price': row['close'],
                    'quantity_usd': position_size,
                    'position_size': position_size,
                    'pnl': 0.0,
                    'cumulative_capital': capital
                })
                position = current_signal
                entry_idx = row['idx']
    
    return pl.DataFrame(trades_list)

In [5]:
def calculate_performance_metrics(df_with_signals, df_trades=None):
    """
    Calcule Sharpe ratio, total return et nombre de trades
    
    Parameters:
    -----------
    df_with_signals : polars DataFrame
        DataFrame avec colonnes: timestamp, close, signal, strategy_returns
    df_trades : polars DataFrame (optional)
        DataFrame des trades. Si None, sera calculé automatiquement
        
    Returns:
    --------
    dict avec les métriques
    """
    # 1. Nombre de trades (changements de signal)
    if df_trades is None:
        # Compte les changements de signal (quand diff != 0)
        signal_changes = df_with_signals['signal'].diff().abs()
        num_trades = signal_changes.filter(signal_changes > 0).len()
    else:
        num_trades = len(df_trades)
    
    # 2. Strategy returns
    strategy_rets = df_with_signals['strategy_returns'].drop_nulls()
    
    if len(strategy_rets) == 0:
        return {
            'sharpe_ratio': np.nan,
            'total_return': np.nan,
            'annualized_return': np.nan,
            'annualized_volatility': np.nan,
            'num_trades': num_trades,
            'max_drawdown': np.nan,
            'win_rate': np.nan
        }
    
    # 3. Sharpe Ratio
    mean_return = strategy_rets.mean()
    std_return = strategy_rets.std()
    sharpe = (mean_return / std_return * np.sqrt(252)) if std_return > 0 else 0
    
    # 4. Total Return (cumulative)
    cumulative_returns = (1 + strategy_rets).product()
    total_return = cumulative_returns - 1
    
    # 5. Annualized Return
    num_years = len(strategy_rets) / 252  # Approximation
    if num_years > 0:
        annualized_return = (1 + total_return) ** (1 / num_years) - 1
    else:
        annualized_return = 0
    
    # 6. Annualized Volatility
    annualized_vol = std_return * np.sqrt(252)
    
    # 7. Max Drawdown
    cumulative = (1 + df_with_signals['strategy_returns'].fill_null(0)).cum_prod()
    running_max = cumulative.cum_max()
    drawdown = (cumulative - running_max) / running_max
    max_dd = drawdown.min()
    
    # 8. Win Rate (% de returns positifs)
    positive_returns = strategy_rets.filter(strategy_rets > 0).len()
    win_rate = positive_returns / len(strategy_rets) if len(strategy_rets) > 0 else 0
    
    return {
        'sharpe_ratio': sharpe,
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_vol,
        'num_trades': num_trades,
        'max_drawdown': abs(max_dd),
        'win_rate': win_rate,
        'num_periods': len(strategy_rets)
    }


def print_performance_summary(metrics):
    """
    Affiche un résumé propre des performances
    """
    print("\n" + "="*60)
    print("           PERFORMANCE SUMMARY")
    print("="*60)
    print(f"Sharpe Ratio:              {metrics['sharpe_ratio']:>10.2f}")
    print(f"Total Return:              {metrics['total_return']:>10.2%}")
    print(f"Annualized Return:         {metrics['annualized_return']:>10.2%}")
    print(f"Annualized Volatility:     {metrics['annualized_volatility']:>10.2%}")
    print(f"Max Drawdown:              {metrics['max_drawdown']:>10.2%}")
    print(f"Win Rate:                  {metrics['win_rate']:>10.2%}")
    print(f"Number of Trades:          {metrics['num_trades']:>10.0f}")
    print(f"Trading Periods:           {metrics['num_periods']:>10.0f}")
    print("="*60 + "\n")

In [6]:
import polars as pl
import numpy as np

def get_trading_signal(history: pl.DataFrame, column: str = 'close'):
    """
    TREND Trading Signal in Polars
    Uses t-statistics of historical daily log-returns to reflect trend strength.
    """

    # Determine which datetime column to use
    if "datetime" in history.columns:
        dtcol = "datetime"
    elif "timestamp" in history.columns:
        dtcol = "timestamp"
    else:
        raise ValueError("No datetime/timestamp column found.")

    # symbols avaible in the dataframe
    symbols = sorted(history['symbol'].unique().to_list())
    
    dict_results = {}
    
    for symbol in symbols:
        hist_symbol = history.filter(pl.col('symbol') == symbol)
        
        # Create a date-only column
        hist_symbol = hist_symbol.with_columns(
            pl.col(dtcol).dt.date().alias("date")
        )

        # Last settlement price per day
        settle_max = (
            hist_symbol.group_by("date", maintain_order=True)
                .agg(pl.col(column).last())
                .sort("date")
        )

        # Prices as numpy array
        prices = settle_max[column].to_numpy()

        # Daily log returns
        log_returns = np.log(prices[1:] / prices[:-1])

        # t-statistic
        mean = np.mean(log_returns)
        std = np.std(log_returns, ddof=1)  # sample std
        n = len(log_returns)
        t_stat = mean / (std / np.sqrt(n))

        # Cap between -1 and 1
        dict_results[symbol] = float(np.clip(t_stat, -1, 1))
    
    return dict_results


In [7]:
import polars as pl
import numpy as np
from datetime import timedelta

def get_y_z_volatility(
        history: pl.DataFrame, 
        available_symbols: list[str], 
        one_month: int = 30,
        data_frequency: str = "1h"  # <--- NOUVEAU PARAMETRE
    ):
        """
        Yang & Zhang Drift-Independent Volatility Estimation (Polars version)
        Annualized per symbol.
        
        Parameters:
        -----------
        data_frequency : str
            '1h' for hourly, '1d' for daily, etc.
        """
        results = []
        
        # order symbols
        available_symbols = sorted(available_symbols)
                
        # Determine annualization factor based on frequency
        annualization_factors = {
            "1h": np.sqrt(252 * 24),      # ~6048 hours per year
            "4h": np.sqrt(252 * 6),       # ~1512 4-hour periods
            "1d": np.sqrt(252),           # 252 days
            "1D": np.sqrt(252),
            "daily": np.sqrt(252),
        }
        
        annualization = annualization_factors.get(data_frequency, np.sqrt(252))
        
        # Make sure datetime column exists
        if "datetime" not in history.columns:
            raise ValueError("history must have a 'datetime' column")
        
        # Ensure datetime type
        history = history.with_columns(
            pl.col("datetime").cast(pl.Datetime("ns"))
        )
        
        # Determine last date across first symbol
        latest_date = history.filter(pl.col("symbol") == available_symbols[0])["datetime"].max()
        cutoff_date = latest_date - timedelta(days=one_month)
        
        for ticker in available_symbols:
            # Select last month of data for ticker
            past_month = (
                history.filter(
                    (pl.col("symbol") == ticker) & 
                    (pl.col("datetime") >= cutoff_date)
                )
                .sort("datetime")
            )
            
            estimation_period = past_month.shape[0]
            
            if estimation_period <= 1:
                results.append(np.nan)
                continue
            
            # Calculate k
            k = 0.34 / (1.34 + (estimation_period + 1) / max(estimation_period - 1, 1))
            
            # Convert to NumPy arrays
            o = past_month["open"].to_numpy()
            h = past_month["high"].to_numpy()
            l = past_month["low"].to_numpy()
            c = past_month["close"].to_numpy()
            
            # sigma__o_j : overnight jump vol (close-to-open)
            oc_log_returns = np.log(o / np.roll(c, 1))[1:]
            sigma_oj = np.std(oc_log_returns[np.isfinite(oc_log_returns)], ddof=1)
            
            # sigma__s_d : standard vol (close-to-close)
            cc_log_returns = np.log(c / np.roll(c, 1))[1:]
            sigma_sd = np.std(cc_log_returns[np.isfinite(cc_log_returns)], ddof=1)
            
            # sigma__r_s : Rogers & Satchell range vol
            H = np.log(h / o)
            L = np.log(l / o)
            C = np.log(c / o)
            sigma_rs_daily = np.sqrt(H * (H - C) + L * (L - C))
            sigma_rs = np.mean(sigma_rs_daily[np.isfinite(sigma_rs_daily)])
            
            # Yang & Zhang period volatility
            sigma_yz = np.sqrt(sigma_oj**2 + k * sigma_sd**2 + (1 - k) * sigma_rs**2)
            
            # Annualize with correct factor
            results.append(sigma_yz * annualization)  # <--- FIX ICI
        
        return results


In [8]:
import polars as pl
import numpy as np
from datetime import timedelta

def get_correlation_factor(
    history: pl.DataFrame, 
    trade_signals: dict, 
    available_symbols: list,
    window: int = 90
):
    """
    Calculate Correlation Factor - VERSION CORRIGÉE
    """
    dtcol = "datetime" if "datetime" in history.columns else "timestamp"
    lookback_date = history[dtcol].max() - timedelta(days=window)
    
    # Collecte returns avec dates communes
    all_returns = []
    
    # sort available_symbols
    available_symbols = sorted(available_symbols)
    
    for symbol in available_symbols:
        hist_sym = history.filter(
            (pl.col("symbol") == symbol) & 
            (pl.col(dtcol) >= lookback_date)
        )
        
        # Daily close avec DATE
        daily = (
            hist_sym
            .with_columns(pl.col(dtcol).dt.date().alias("date"))
            .group_by("date", maintain_order=True)
            .agg(pl.col("close").last())
            .sort("date")
            .with_columns(pl.col("close").pct_change().alias("return"))
            .select(['date', 'return'])
            .rename({'return': symbol})
        )
        
        all_returns.append(daily)
    
    # Merge sur les dates COMMUNES (inner join)
    returns_df = all_returns[0]
    for df in all_returns[1:]:
        returns_df = returns_df.join(df, on='date', how='inner')  # ← FIX : inner join
    
    # Drop dates, convert to pandas
    import pandas as pd
    returns_pd = returns_df.drop('date').to_pandas()
    
    # Vérifie qu'on a assez de données
    if len(returns_pd) < 10:
        # Pas assez de données → CF = sqrt(N) par défaut
        return np.sqrt(len(available_symbols))
    
    corr_matrix = returns_pd.corr()
    
    n_assets = len(available_symbols)
    
    # Calculate rho_bar
    summation = 0
    count = 0
    for i in range(n_assets - 1):
        for j in range(i + 1, n_assets):
            symbol_i = available_symbols[i]
            symbol_j = available_symbols[j]
            
            x_i = trade_signals[symbol_i]
            x_j = trade_signals[symbol_j]
            rho_ij = corr_matrix.loc[symbol_i, symbol_j]
            
            if not np.isnan(rho_ij):
                summation += x_i * x_j * rho_ij
                count += 1
    
    if count == 0:
        return np.sqrt(n_assets)
    
    rho_bar = (2 * summation) / (n_assets * (n_assets - 1))
    
    # Correlation factor
    cf = np.sqrt(n_assets / (1 + (n_assets - 1) * rho_bar))
    
    return cf


In [9]:
def rebalance_portfolio_correct(
        history: pl.DataFrame,
        current_positions: dict,
        portfolio_target_sigma: float = 0.12,
        capital: float = 100000,
        window: int = 90,
        data_frequency: str = None  # Auto-detect si None
    ):
        """
        Rebalance portfolio - VERSION CORRECTE
        
        Returns positions in NUMBER OF CONTRACTS
        
        Parameters:
        -----------
        data_frequency : str
            Fréquence des données ('1h', '1d', '4h', etc.) pour le calcul correct de la volatilité annualisée
        """

        if data_frequency is None:
            # Détecte la fréquence en regardant l'écart moyen entre timestamps
            dtcol = "datetime" if "datetime" in history.columns else "timestamp"
            sample = history.sort(dtcol).head(100)
            time_diffs = sample[dtcol].diff().drop_nulls()
            avg_diff_seconds = time_diffs.mean().total_seconds()
            
            if avg_diff_seconds < 3600 * 2:  # < 2 heures
                data_frequency = "1h"
            elif avg_diff_seconds < 3600 * 12:  # < 12 heures
                data_frequency = "4h"
            else:
                data_frequency = "1d"
        
            
        available_symbols = sorted(history['symbol'].unique().to_list())
        
        if len(available_symbols) == 0:
            return current_positions
        
        # 1-3. Same as before
        trade_signals = get_trading_signal(history)
        volatility = get_y_z_volatility(history, available_symbols, data_frequency=data_frequency)
        c_f_rho_bar = get_correlation_factor(history, trade_signals, available_symbols, window=window)
        
        # 4. Calculate weights
        n_assets = len(available_symbols)
        new_positions = {}
        
        # Contract multipliers (standard pour futures)
        contract_multipliers = {
            'CL=F': 1000,   # 1000 barrels per contract
            'RB=F': 42000,  # 42000 gallons per contract
            'NG=F': 10000,  # 10000 MMBtu per contract
            'HG=F': 25000,  # 25000 pounds per contract
            'GC=F': 100,    # 100 troy ounces per contract
        }
        
        for i, symbol in enumerate(available_symbols):
            signal = trade_signals[symbol]
            vol = volatility[i]
            
            if np.isnan(vol) or vol == 0:
                new_positions[symbol] = 0
                continue
            
            # Baltas & Kosowski weight (-1 to +1)
            weight = (signal * portfolio_target_sigma * c_f_rho_bar) / (n_assets * vol)
            weight = np.clip(weight, -1, 1)
            
            # Get last price
            last_price = history.filter(pl.col('symbol') == symbol)['close'].tail(1)[0]
            
            # Get contract multiplier
            multiplier = contract_multipliers.get(symbol, 1)
            
            # Calculate dollar value to allocate
            dollar_allocation = weight * capital
            
            # Calculate number of contracts
            # 1 contract = last_price * multiplier dollars
            contract_value = last_price * multiplier
            num_contracts = dollar_allocation / contract_value
            
            new_positions[symbol] = round(num_contracts)  # Round to whole contracts
        
        return new_positions, {
            'weights': {symbol: (signal * portfolio_target_sigma * c_f_rho_bar) / (n_assets * volatility[i]) 
                    for i, symbol in enumerate(available_symbols)},
            'signals': trade_signals,
            'volatilities': dict(zip(available_symbols, volatility)),
            'correlation_factor': c_f_rho_bar
        }


In [10]:

# Usage
# Note: passez data_frequency="1h" si vos données sont horaires, "1d" si quotidiennes, etc.
new_positions, details = rebalance_portfolio_correct(
    history=df,
    current_positions={},
    portfolio_target_sigma=0.12,
    capital=initial,
    window=window_days,
)


print("\n=== PORTFOLIO REBALANCE ===")
print(f"Capital: ${1000:,.0f}")
print(f"\nNew Positions (number of contracts):")
for symbol, contracts in new_positions.items():
    direction = "LONG" if contracts > 0 else "SHORT"
    print(f"  {symbol}: {abs(contracts)} contracts {direction}")

print(f"\nDetails:")
print(f"  Signals: {details['signals']}")
print(f"  Volatilities: {details['volatilities']}")
print(f"  Correlation Factor: {details['correlation_factor']:.4f}")
print(f"  Weights: {details['weights']}")



=== PORTFOLIO REBALANCE ===
Capital: $1,000

New Positions (number of contracts):
  CL=F: 0 contracts SHORT
  RB=F: 0 contracts SHORT

Details:
  Signals: {'CL=F': -0.41837985056458094, 'RB=F': 0.3820708286804292}
  Volatilities: {'CL=F': np.float64(0.2574283254831564), 'RB=F': np.float64(1.4078284825026575)}
  Correlation Factor: 1.4252
  Weights: {'CL=F': np.float64(0.12691161368200152), 'RB=F': np.float64(0.023206409445875965)}


In [11]:
def backtest_baltas_kosowski_fractional(
        history: pl.DataFrame, 
        initial_capital=10000,
        portfolio_target_sigma=0.43
    ):
        """
        Backtest avec positions fractionnaires (pas de contrainte sur les contrats entiers)
        """
        results = []
        capital = initial_capital
        
        history_with_month = history.with_columns(
                pl.col('datetime').dt.truncate('1mo').alias('month')
            )

        months = history_with_month['month'].unique().sort()

        for i, month in enumerate(months[1:]):
    
            hist_until_month = history.filter(pl.col('datetime') < month)

            # Calcule seulement les weights (pas les positions en contrats)
            _, details = rebalance_portfolio_correct(
                hist_until_month, 
                {}, 
                capital=capital,
                portfolio_target_sigma=portfolio_target_sigma
            )

            weights = details['weights']

            # Returns du mois
            if i+1 < len(months) - 1:
                next_month = months[i+2]
            else:
                next_month = month + timedelta(days=60)
                
            month_data = history.filter(
                (pl.col('datetime') >= month) & 
                (pl.col('datetime') < next_month)
            )

            month_returns = {}
            for symbol in weights.keys():
                symbol_data = month_data.filter(pl.col('symbol') == symbol)
                if len(symbol_data) > 1:
                    start_price = symbol_data['close'].head(1)[0]
                    end_price = symbol_data['close'].tail(1)[0]
                    month_returns[symbol] = (end_price - start_price) / start_price
                else:
                    month_returns[symbol] = 0
                    
            # Portfolio return = Σ(weight_i × return_i)
            portfolio_return = sum(
                weights[symbol] * month_returns[symbol] 
                for symbol in weights.keys()
            )

            capital *= (1 + portfolio_return)

            results.append({
                'month': month,
                'weights': weights,
                'returns': month_returns,
                'portfolio_return': portfolio_return,
                'capital': capital
            })

        return pl.DataFrame(results)

In [12]:
# TESTE CETTE VERSION
backtest_frac = backtest_baltas_kosowski_fractional(
    df, 
    initial_capital=initial,
    portfolio_target_sigma=0.43
)

# Performance finale (CORRIGÉE)
final = backtest_frac['capital'].tail(1)[0]
total_return = (final - initial) / initial

print(f"\nInitial Capital: ${initial:,.0f}")
print(f"Final Capital: ${final:,.2f}")
print(f"Total Return: {total_return:.2%}")

# Calcule aussi le Sharpe
monthly_returns = backtest_frac['portfolio_return']
sharpe = (monthly_returns.mean() / monthly_returns.std()) * np.sqrt(12)
print(f"Sharpe Ratio: {sharpe:.2f}")

backtest_frac


Initial Capital: $1,000
Final Capital: $851.39
Total Return: -14.86%
Sharpe Ratio: -0.81


month,weights,returns,portfolio_return,capital
datetime[μs],struct[2],struct[2],f64,f64
2024-12-01 00:00:00,"{0.065512,0.022909}","{0.052424,0.132834}",0.006477,1006.477449
2025-01-01 00:00:00,"{0.623851,0.199836}","{0.026279,-0.147614}",-0.013104,993.288354
2025-02-01 00:00:00,"{-0.281807,-0.079534}","{-0.05473,0.152384}",0.003304,996.569703
2025-03-01 00:00:00,"{0.44761,0.166272}","{0.017239,0.099814}",0.024313,1020.79884
2025-04-01 00:00:00,"{0.670737,0.258896}","{-0.184708,-0.186954}",-0.172292,844.923082
…,…,…,…,…
2025-07-01 00:00:00,"{0.041196,0.036414}","{0.06757,-0.104046}",-0.001005,847.106484
2025-08-01 00:00:00,"{-0.144776,-0.075676}","{-0.077643,-0.030724}",0.013566,858.598356
2025-09-01 00:00:00,"{-0.206131,-0.117048}","{-0.024074,0.112558}",-0.008212,851.547242


In [13]:
def create_trades_from_backtest(backtest_results: pl.DataFrame, symbol_to_plot: str = 'CL=F', initial_capital: float = 100000):
    """
    Crée des trades pour UN SEUL symbol (pour plotting)
    VERSION CORRIGÉE
    """
    trades_list = []
    
    for i, row in enumerate(backtest_results.iter_rows(named=True)):
        month = row['month']
        weights = row['weights']
        portfolio_return = row['portfolio_return']
        capital = row['capital']
        
        # Weight pour le symbol spécifique
        if symbol_to_plot not in weights:
            continue
            
        weight = weights[symbol_to_plot]
        
        # Skip si weight trop petit
        if abs(weight) < 0.001:
            continue
        
        # Action
        action = "BUY" if weight > 0 else "SELL"
        
        # IMPORTANT: Trouve le VRAI prix du symbol à cette date
        month_data = df.filter(
            (pl.col('symbol') == symbol_to_plot) &  # ← FILTRE LE BON SYMBOL
            (pl.col('datetime') >= month)
        )
        
        if len(month_data) == 0:
            continue
            
        price = month_data['close'].head(1)[0]  # ← PRIX RÉEL, pas weight !
        
        trades_list.append({
            'timestamp': month,
            'position_number': i,
            'action': action,
            'price': price,  # ← Vrai prix CL (55-75)
            'quantity_usd': abs(weight) * capital,
            'position_size': abs(weight),
            'pnl': portfolio_return * capital if i > 0 else 0.0,
            'cumulative_capital': capital / initial_capital
        })
    
    return pl.DataFrame(trades_list)

In [14]:
# UTILISE CETTE VERSION
trades_bk = create_trades_from_backtest(
    backtest_frac, 
    symbol_to_plot='CL=F',  # ← Spécifie CL uniquement
    initial_capital=initial
)

print(trades_bk)

shape: (12, 8)
┌────────────┬────────────┬────────┬───────────┬────────────┬────────────┬────────────┬────────────┐
│ timestamp  ┆ position_n ┆ action ┆ price     ┆ quantity_u ┆ position_s ┆ pnl        ┆ cumulative │
│ ---        ┆ umber      ┆ ---    ┆ ---       ┆ sd         ┆ ize        ┆ ---        ┆ _capital   │
│ datetime[μ ┆ ---        ┆ str    ┆ f64       ┆ ---        ┆ ---        ┆ f64        ┆ ---        │
│ s]         ┆ i64        ┆        ┆           ┆ f64        ┆ f64        ┆            ┆ f64        │
╞════════════╪════════════╪════════╪═══════════╪════════════╪════════════╪════════════╪════════════╡
│ 2024-12-01 ┆ 0          ┆ BUY    ┆ 68.290001 ┆ 65.935928  ┆ 0.065512   ┆ 0.0        ┆ 1.006477   │
│ 00:00:00   ┆            ┆        ┆           ┆            ┆            ┆            ┆            │
│ 2025-01-01 ┆ 1          ┆ BUY    ┆ 71.919998 ┆ 619.663889 ┆ 0.623851   ┆ -13.016263 ┆ 0.993288   │
│ 00:00:00   ┆            ┆        ┆           ┆            ┆            ┆  

In [15]:
from quanta.clients.chart import ChartClient
chart_client = ChartClient()
chart_client.plot(
    df_cl, 
    "cl=F",  
    trades_df=trades_bk, 
    theme='professional'
)

Plotting 5529 bars for cl=F with x_axis_type='row_nb'
With 12 trades


Exception: The (row, col) pair sent is out of range. Use Figure.print_grid to view the subplot grid. 

# old test

In [16]:
import polars as pl

def momentum_strategy(df, lookback_period=252):
    # 1. Calcule returns 12-month (252 jours de trading)
    df = df.with_columns([
        (pl.col('close').pct_change(lookback_period)).alias('returns_12m')
    ])

    # 2. Signal simple (+1 si positif, -1 si négatif)
    df = df.with_columns([
        pl.when(pl.col('returns_12m') > 0).then(1)
        .when(pl.col('returns_12m') < 0).then(-1)
        .otherwise(0)
        .alias('signal')
    ])


    # 3. Daily returns
    df = df.with_columns([
        pl.col('close').pct_change().alias('daily_returns')
    ])

    # 4. Strategy returns (signal décalé d'un jour)
    df = df.with_columns([
        (pl.col('signal').shift(1) * pl.col('daily_returns')).alias('strategy_returns')
    ])
    return df

df_cl = momentum_strategy(df_cl, lookback_period=252)
df_cl


timestamp,datetime,open,high,low,close,volume,symbol,returns_12m,signal,daily_returns,strategy_returns
i64,datetime[μs],f64,f64,f64,f64,i64,str,f64,i32,f64,f64
1732489200,2024-11-25 00:00:00,71.440002,71.480003,71.07,71.279999,1788,"""CL=F""",null,0,null,null
1732492800,2024-11-25 01:00:00,71.290001,71.360001,71.199997,71.260002,1270,"""CL=F""",null,0,-0.000281,-0.0
1732496400,2024-11-25 02:00:00,71.269997,71.43,71.129997,71.160004,2350,"""CL=F""",null,0,-0.001403,-0.0
1732500000,2024-11-25 03:00:00,71.160004,71.199997,70.900002,70.949997,2783,"""CL=F""",null,0,-0.002951,-0.0
1732503600,2024-11-25 04:00:00,70.949997,71.040001,70.830002,71.019997,2626,"""CL=F""",null,0,0.000987,0.0
…,…,…,…,…,…,…,…,…,…,…,…
1763485200,2025-11-18 18:00:00,60.07,60.400002,59.779999,60.299999,23849,"""CL=F""",-0.010827,-1,0.005503,-0.005503
1763488800,2025-11-18 19:00:00,60.290001,60.919998,60.290001,60.630001,44163,"""CL=F""",-0.005903,-1,0.005473,-0.005473
1763492400,2025-11-18 20:00:00,60.630001,60.869999,60.57,60.740002,29320,"""CL=F""",-0.005404,-1,0.001814,-0.001814


In [17]:
trades = create_trades_with_sizing(
    df_cl, 
    initial_capital=1.0, 
    position_size=0.519246)
trades

/var/folders/kw/gf0t96sj4_d256ws4dzdlxc80000gn/T/ipykernel_12605/1518853554.py:11: DeprecationWarning:

`DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.



timestamp,position_number,action,price,quantity_usd,position_size,pnl,cumulative_capital
datetime[μs],i64,str,f64,f64,f64,f64,f64
2024-12-11 11:00:00,0,"""SELL""",69.25,0.519246,0.519246,0.0,1.0
2024-12-12 03:00:00,0,"""BUY""",70.260002,0.519246,0.519246,-0.007573,0.992427
2024-12-12 03:00:00,1,"""BUY""",70.260002,0.519246,0.519246,0.0,0.992427
2024-12-18 22:00:00,1,"""SELL""",70.0,0.519246,0.519246,-0.001922,0.990505
2024-12-18 22:00:00,2,"""SELL""",70.0,0.519246,0.519246,0.0,0.990505
…,…,…,…,…,…,…,…
2025-11-14 12:00:00,144,"""SELL""",59.560001,0.519246,0.519246,0.0,0.794815
2025-11-14 15:00:00,144,"""BUY""",60.040001,0.519246,0.519246,-0.004185,0.79063
2025-11-14 15:00:00,145,"""BUY""",60.040001,0.519246,0.519246,0.0,0.79063


In [18]:
metrics = calculate_performance_metrics(df_cl)
print_performance_summary(metrics)


           PERFORMANCE SUMMARY
Sharpe Ratio:                   -0.27
Total Return:                 -36.89%
Annualized Return:             -2.08%
Annualized Volatility:          6.87%
Max Drawdown:                  49.00%
Win Rate:                      45.75%
Number of Trades:                 147
Trading Periods:                 5528



In [19]:
df_cl

timestamp,datetime,open,high,low,close,volume,symbol,returns_12m,signal,daily_returns,strategy_returns
i64,datetime[μs],f64,f64,f64,f64,i64,str,f64,i32,f64,f64
1732489200,2024-11-25 00:00:00,71.440002,71.480003,71.07,71.279999,1788,"""CL=F""",null,0,null,null
1732492800,2024-11-25 01:00:00,71.290001,71.360001,71.199997,71.260002,1270,"""CL=F""",null,0,-0.000281,-0.0
1732496400,2024-11-25 02:00:00,71.269997,71.43,71.129997,71.160004,2350,"""CL=F""",null,0,-0.001403,-0.0
1732500000,2024-11-25 03:00:00,71.160004,71.199997,70.900002,70.949997,2783,"""CL=F""",null,0,-0.002951,-0.0
1732503600,2024-11-25 04:00:00,70.949997,71.040001,70.830002,71.019997,2626,"""CL=F""",null,0,0.000987,0.0
…,…,…,…,…,…,…,…,…,…,…,…
1763485200,2025-11-18 18:00:00,60.07,60.400002,59.779999,60.299999,23849,"""CL=F""",-0.010827,-1,0.005503,-0.005503
1763488800,2025-11-18 19:00:00,60.290001,60.919998,60.290001,60.630001,44163,"""CL=F""",-0.005903,-1,0.005473,-0.005473
1763492400,2025-11-18 20:00:00,60.630001,60.869999,60.57,60.740002,29320,"""CL=F""",-0.005404,-1,0.001814,-0.001814


In [50]:
trades

timestamp,position_number,action,price,quantity_usd,position_size,pnl,cumulative_capital
datetime[μs],i64,str,f64,f64,f64,f64,f64
2024-11-20 06:00:00,0,"""SELL""",68.870003,0.519246,0.519246,0.0,1.0
2024-12-12 06:00:00,0,"""BUY""",70.019997,0.519246,0.519246,-0.00867,0.99133
2024-12-12 06:00:00,1,"""BUY""",70.019997,0.519246,0.519246,0.0,0.99133
2024-12-16 06:00:00,1,"""SELL""",70.709999,0.519246,0.519246,0.005117,0.996446
2024-12-16 06:00:00,2,"""SELL""",70.709999,0.519246,0.519246,0.0,0.996446
…,…,…,…,…,…,…,…
2025-01-24 06:00:00,4,"""SELL""",74.660004,0.519246,0.519246,0.0,1.010315
2025-02-04 06:00:00,4,"""BUY""",72.699997,0.519246,0.519246,0.013631,1.023946
2025-02-04 06:00:00,5,"""BUY""",72.699997,0.519246,0.519246,0.0,1.023946


In [51]:
from quanta.clients.chart import ChartClient
chart_client = ChartClient()
chart_client.plot(
    df_cl, 
    "cl=F",  
    trades_df=trades, 
    theme='professional'
)

Plotting 501 bars for cl=F with x_axis_type='row_nb'
With 13 trades
Debug: 13 trades after processing
shape: (5, 4)
┌─────────────────────┬─────────┬────────┬───────────┐
│ timestamp           ┆ x_value ┆ action ┆ price     │
│ ---                 ┆ ---     ┆ ---    ┆ ---       │
│ datetime[μs]        ┆ i64     ┆ str    ┆ f64       │
╞═════════════════════╪═════════╪════════╪═══════════╡
│ 2024-11-20 06:00:00 ┆ 252     ┆ SELL   ┆ 68.870003 │
│ 2024-12-12 06:00:00 ┆ 267     ┆ BUY    ┆ 70.019997 │
│ 2024-12-12 06:00:00 ┆ 267     ┆ BUY    ┆ 70.019997 │
│ 2024-12-16 06:00:00 ┆ 269     ┆ SELL   ┆ 70.709999 │
│ 2024-12-16 06:00:00 ┆ 269     ┆ SELL   ┆ 70.709999 │
└─────────────────────┴─────────┴────────┴───────────┘


In [ ]:
# Regarde les composantes
signals = get_trading_signal(df)
vols = get_y_z_volatility(df, ['CL=F', 'RB=F'])

print("Signals:", signals)
print("Volatilities:", vols)

# Correlation factor
cf = get_correlation_factor(df, signals, ['CL=F', 'RB=F'])
print("Correlation Factor:", cf)

# Calcul manuel du weight pour CL
signal_cl = signals['CL=F']
vol_cl = vols[0]
target_sigma = 0.12
n_assets = 2

weight_cl = (signal_cl * target_sigma * cf) / (n_assets * vol_cl)
print(f"\nWeight CL=F: {weight_cl:.4f}")
print(f"Capped: {np.clip(weight_cl, -1, 1):.4f}")

Signals: {'RB=F': -0.14073238390788778, 'CL=F': -0.449233775441133}
Volatilities: [np.float64(0.05233710367508116), np.float64(0.39922424440135446)]
Correlation Factor: 1.3880856506051245

Weight CL=F: -0.7149
Capped: -0.7149


In [28]:
# Momentum simple (ton ancien code)
df_cl_simple = momentum_strategy(df.filter(pl.col('symbol') == 'CL=F'), lookback_period=252)
last_signal = df_cl_simple['signal'].tail(1)[0]

print(f"\nMomentum simple CL=F: {last_signal}")
print(f"Baltas & Kosowski CL=F: {signals['CL=F']:.4f}")


Momentum simple CL=F: 0


NameError: name 'signals' is not defined